# Python Multivariate Analysis Demo

*Author: Evan Carey*

*Copyright 2017, BH Analytics, LLC*

## Overview

The purpose of this section is to demonstrate functionality of Python that you will be learning in this course. This is a demo that goes over various aspects of multivariate binary prediction using both classical (frequentist) and machine learning approaches. 

The data used to support is a synthetic dataset representing sales data. There are two files. We will fit a model on the first file, then use the results to make prediction on the second file. 

## Libraries

In [1]:
## Import Modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.metrics import confusion_matrix
import sklearn
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels

In [2]:
## Enable inline plotting for graphics
%matplotlib inline

In [3]:
## Get Version information
print(sys.version)
print("Pandas version: {0}".format(pd.__version__))
print("Matplotlib version: {0}".format(matplotlib.__version__))
print("Numpy version: {0}".format(np.__version__))
print("SciKitLearn version: {0}".format(sklearn.__version__))
print("Statsmodels version: {0}".format(statsmodels.__version__))

3.6.0 |Anaconda custom (64-bit)| (default, Dec 23 2016, 11:57:41) [MSC v.1900 64 bit (AMD64)]
Pandas version: 0.19.2
Matplotlib version: 2.0.0
Numpy version: 1.11.3
SciKitLearn version: 0.18.1
Statsmodels version: 0.6.1


## Check your working directory

Subsequent sessions may require you to identify and update your working directory so paths correctly point at the downloaded data files. You can check your working directory like so:

In [4]:
## Check/set your working directory
print("My working directory:\n" + os.getcwd())
# Set Working Directory (if needed)
os.chdir(r"C:\Users\evan\Dropbox\BH_Katie_Share\Goldman_Sachs")
print("My new working directory:\n" + os.getcwd())

My working directory:
C:\Users\evan\Dropbox\BH_Katie_Share\Goldman_Sachs\Intermediate_python\notebooks
My new working directory:
C:\Users\evan\Dropbox\BH_Katie_Share\Goldman_Sachs


## Import the dataset

You should already have some experience with Pandas if you are taking this course. Below, I use Pandas to import a CSV file that we will then use to fit predictive models.

In [5]:
## Import Sales Data File
customer_sales = pd.read_csv('data/simulated_sales_data/customer_sales1.csv')
customer_sales.head()

customer_id sale  sale_amount  region  age  gender activity  \
0       100000   No     0.000000      35   62    Male      Med   
1       100001  Yes   884.478029      13   71  Female     High   
2       100002   No     0.000000      23   53    Male     High   
3       100003   No     0.000000      12   60    Male      Med   
4       100004  Yes   243.937661      44   62    Male      Med   

   marketing_exposure         x1         x2          x3  x4  x5  num_accounts  \
0                   6  73.674643   8.263063   93.879407  10   1             0   
1                   5  49.195984  11.925201   72.862514   7   2             2   
2                   6  49.868492  10.932390  128.150580  12   2             1   
3                   7  42.563154   9.725310  126.266391  11   1             0   
4                  11  45.035440   9.941171  108.037634   9   3             0   

  current_customer  income  
0               No      78  
1               No      66  
2              Yes      86  
3              Yes      70  
4               No      59

In [6]:
customer_sales.tail()

customer_id sale  sale_amount  region  age  gender activity  \
7995       109995   No      0.00000       5   57    Male      Low   
7996       109996  Yes    722.51420      20   75  Female     High   
7997       109997  Yes    136.01130      11   58    Male      Low   
7998       109998  Yes    357.46150      49   64    Male      Med   
7999       109999  Yes     19.83436      30   46    Male      Med   

      marketing_exposure        x1         x2         x3  x4  x5  \
7995                   4  70.11600  10.037210   87.65496  13   0   
7996                   1  38.41591  14.850920  129.32720  12   1   
7997                   6  49.01480  10.558640   97.09751  15   2   
7998                   7  52.21514   8.967199   59.17881  11   0   
7999                   4  21.09239   9.611086  106.87850   9   5   

      num_accounts current_customer  income  
7995             1               No      59  
7996             0               No      74  
7997             0              Yes      54  
7998             0              Yes      59  
7999             0              Yes      59

## Investigate and change dtypes

In [7]:
customer_sales.dtypes

customer_id             int64
sale                   object
sale_amount           float64
region                  int64
age                     int64
gender                 object
activity               object
marketing_exposure      int64
x1                    float64
x2                    float64
x3                    float64
x4                      int64
x5                      int64
num_accounts            int64
current_customer       object
income                  int64
dtype: object

In [8]:
## Change dtypes
customer_sales['customer_id'] = \
              customer_sales.customer_id.astype('object')
customer_sales['region'] = \
              customer_sales.region.astype('object')
customer_sales['current_customer'] = \
              customer_sales.current_customer.astype('category')
customer_sales['gender'] = \
              customer_sales.gender.astype('category')
customer_sales.activity.unique()

array(['Med', 'High', 'Low'], dtype=object)

In [9]:
customer_sales['activity'] = \
              pd.Categorical(customer_sales.activity,
                             ['Low','Med','High'],
                             ordered=True)
customer_sales['sale'] = \
              customer_sales.sale.astype('category')

## Dataset description

The next step should always be to examine the dataset for any issues. This dataset is very 'clean', since it is simulated data and the aim of this demo is not to show data cleaning.

In [10]:
## describe dataset (numeric)
customer_sales.describe()

sale_amount          age  marketing_exposure           x1           x2  \
count  8000.000000  8000.000000         8000.000000  8000.000000  8000.000000   
mean    248.977381    59.125875            4.791375    49.878415     9.946172   
std     314.309697    11.204908            2.594888    10.062941     2.015181   
min       0.000000    21.000000            0.000000    14.596822     2.011882   
25%       0.000000    51.000000            3.000000    43.282805     8.580155   
50%     118.774700    59.000000            4.000000    49.848510     9.964735   
75%     418.922000    67.000000            6.000000    56.762955    11.294125   
max    2111.991000   100.000000           16.000000    89.657440    17.216920   

                x3           x4           x5  num_accounts       income  
count  8000.000000  8000.000000  8000.000000    8000.00000  8000.000000  
mean     99.788263     9.953875     1.966125       0.48925    64.792250  
std      19.861677     3.127174     1.402845       0.69620     9.940457  
min      30.617160     0.000000     0.000000       0.00000    26.000000  
25%      86.244105     8.000000     1.000000       0.00000    58.000000  
50%      99.782738    10.000000     2.000000       0.00000    65.000000  
75%     113.334475    12.000000     3.000000       1.00000    71.000000  
max     168.373700    22.000000     9.000000       4.00000   103.000000

In [11]:
## describe object dtypes
customer_sales.\
    select_dtypes(include=['object']).\
    describe()

customer_id  region
count          8000    8000
unique         8000      50
top          109999      36
freq              1     189

In [12]:
## describe categories
customer_sales.\
    select_dtypes(include=['category']).\
    describe()

sale gender activity current_customer
count   8000   8000     8000             8000
unique     2      2        3                2
top      Yes   Male      Med               No
freq    4765   4804     3587             5647

I think we need a better describe for the categories. Let's write a new function for this:

In [13]:
## Make better categorical describe function
def describe_cats(df,normalize=False): 
    for col in df.\
    select_dtypes(include=['category']).\
    columns:
        print('\n' + col)
        print(df[col].\
              sort_values().\
              value_counts(sort=False,normalize=normalize))

Nice! Now we can get the frequency tables for each of these vars...

In [14]:
## Describe the cats
describe_cats(customer_sales)


sale
No     3235
Yes    4765
Name: sale, dtype: int64

gender
Female    3196
Male      4804
Name: gender, dtype: int64

activity
Low     2808
Med     3587
High    1605
Name: activity, dtype: int64

current_customer
No     5647
Yes    2353
Name: current_customer, dtype: int64


In [15]:
## Describe the cats
describe_cats(customer_sales, True)


sale
No     0.404375
Yes    0.595625
Name: sale, dtype: float64

gender
Female    0.3995
Male      0.6005
Name: gender, dtype: float64

activity
Low     0.351000
Med     0.448375
High    0.200625
Name: activity, dtype: float64

current_customer
No     0.705875
Yes    0.294125
Name: current_customer, dtype: float64


## Multivariate modeling

The main goal of this demo is to show both classical (frequentist) approaches to multivariate analysis, and machine learning approaches. We will begin with  logistic regression.

## Classical stats: logistic regression

We implement a logistic regression using the statsmodel package. This is a form of a general linear model, where the link function is the log odds, and the family is the binomial distribution. We first need to specify a formula for the model.

In [16]:
#### Multivariate modeling
## Use Stats models to fit logistic regression
vars_remove = ['sale','sale_amount','customer_id','region']
vars_left = set(customer_sales.columns) - set(vars_remove)
formula = "sale ~ " + " + ".join(vars_left)
formula

'sale ~ x1 + x2 + income + gender + activity + x3 + age + x4 + x5 + current_customer + num_accounts + marketing_exposure'

We can then fit the model like so:

In [17]:
mod1 = smf.glm(formula=formula,
               data=customer_sales, 
               family=sm.families.Binomial()).fit()
mod1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['sale[No]', 'sale[Yes]']   No. Observations:                 8000
Model:                                   GLM   Df Residuals:                     7986
Model Family:                       Binomial   Df Model:                           13
Link Function:                         logit   Scale:                             1.0
Method:                                 IRLS   Log-Likelihood:                -5160.3
Date:                       Sun, 21 May 2017   Deviance:                       10321.
Time:                               19:45:21   Pearson chi2:                 8.01e+03
No. Iterations:                            6                                         
===========================================================================================
                              coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------
Intercept                   2.7215      0.316      8.624      0.000         2.103     3.340
gender[T.Male]              0.1365      0.062      2.214      0.027         0.016     0.257
activity[T.Med]            -0.5241      0.052    -10.048      0.000        -0.626    -0.422
activity[T.High]           -0.8584      0.067    -12.734      0.000        -0.991    -0.726
current_customer[T.Yes]    -0.1285      0.052     -2.481      0.013        -0.230    -0.027
x1                         -0.0017      0.002     -0.715      0.475        -0.006     0.003
x2                         -0.0027      0.012     -0.233      0.815        -0.026     0.020
income                      0.0006      0.002      0.238      0.812        -0.004     0.005
x3                         -0.0159      0.001    -13.110      0.000        -0.018    -0.013
age                        -0.0178      0.002     -7.619      0.000        -0.022    -0.013
x4                         -0.0037      0.008     -0.486      0.627        -0.018     0.011
x5                         -0.0073      0.017     -0.433      0.665        -0.040     0.026
num_accounts               -0.0161      0.034     -0.475      0.635        -0.082     0.050
marketing_exposure         -0.0006      0.011     -0.060      0.952        -0.022     0.020
===========================================================================================
"""

## Machine Learning Prediction

We will use the scikitlearn package to implement a machine learning approach to inference. We must first create numpy array of the design matrices for the regression. We can use patsy to easily do that:

In [18]:
## use Patsy to create model matrices
Y,X = dmatrices(formula,
                customer_sales.dropna(),
                return_type="dataframe")

We next need to split the dataset apart into testing and training datasets.

In [19]:
## Split Data into training and sample
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X,
                     Y['sale[Yes]'],
                     test_size=0.25,
                     random_state=42)

Let's confirm the dimensionality of these objects...

In [20]:
## Confirm dimensions
X_train.shape

(6000, 14)

In [21]:
y_train.shape

(6000,)

In [22]:
X_test.shape

(2000, 14)

In [23]:
y_test.shape

(2000,)

## Null information rate

An initial concept in machine learning is the 'null information rate'. This is a silly baseline model that we know is not very good. However, it provides a useful comparison when we develop future models. What is a reasonable 'dumb' model for this binary classification problem?

In [24]:
## Null information rate
y_test.mean()

0.5995

Scikitlearn has a built in dummy classifier that works similarly:

In [25]:
## Dummy classifier
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent',
                      random_state=0)
clf.fit(X_train, y_train)
DummyClassifier(constant=None,
                random_state=0,
                strategy='most_frequent')
clf.score(X_test, y_test)  

0.59950000000000003

## Penalized logistic regression

We will now use the scikitlearn package to implement a penalized logistic regression on our test dataset. We can then evaluate the results, on both the in sample data and the test dataset.

In [26]:
## Logistic Regression with l2 penalty
from sklearn import linear_model
clf = linear_model.LogisticRegression()
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

array([[ 733, 1701],
       [ 532, 3034]])

In [27]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,clf.predict(X_train))

0.62783333333333335

In [28]:
## Get kappa
sklearn.metrics.cohen_kappa_score(y_train,clf.predict(X_train))

0.16450382203020963

In [29]:
## get classification metrics
print(sklearn.metrics.classification_report(y_train,clf.predict(X_train)))

             precision    recall  f1-score   support

        0.0       0.58      0.30      0.40      2434
        1.0       0.64      0.85      0.73      3566

avg / total       0.62      0.63      0.60      6000



Let's evaluate it against the test dataset to see if we overfit...

In [30]:
## get classification metrics on test
print(sklearn.metrics.classification_report(y_test,clf.predict(X_test)))

             precision    recall  f1-score   support

        0.0       0.60      0.31      0.41       801
        1.0       0.65      0.86      0.74      1199

avg / total       0.63      0.64      0.61      2000



## Logistic regression with L1 penalty

A different form of penalized logistic regression is to implement an L1 penalty. We can easily do so like this:

In [31]:
## Logistic Regression with l1 penalty
clf = linear_model.LogisticRegression(penalty='l1')
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

array([[ 733, 1701],
       [ 533, 3033]])

In [32]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,clf.predict(X_train))

0.62766666666666671

In [33]:
## Get kappa
sklearn.metrics.cohen_kappa_score(y_train,clf.predict(X_train))

0.16418866439294055

In [34]:
## get classification metrics
print(sklearn.metrics.classification_report(y_train,clf.predict(X_train)))

             precision    recall  f1-score   support

        0.0       0.58      0.30      0.40      2434
        1.0       0.64      0.85      0.73      3566

avg / total       0.62      0.63      0.60      6000



In [35]:
## get classification metrics on test
print(sklearn.metrics.classification_report(y_test,clf.predict(X_test)))

             precision    recall  f1-score   support

        0.0       0.60      0.32      0.41       801
        1.0       0.65      0.86      0.74      1199

avg / total       0.63      0.64      0.61      2000



## Support vector machine

Another reasonable choice for a classifer would be a support vector machine. We can easily implement this as well:

In [36]:
## Support Vector Machine (classification)
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

array([[2426,    8],
       [   0, 3566]])

In [37]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,clf.predict(X_train))

0.9986666666666667

In [38]:
## Get kappa
sklearn.metrics.cohen_kappa_score(y_train,clf.predict(X_train))

0.99723346638823041

In [39]:
## get classification metrics
print(sklearn.metrics.classification_report(y_train,clf.predict(X_train)))

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2434
        1.0       1.00      1.00      1.00      3566

avg / total       1.00      1.00      1.00      6000



In [40]:
## get classification metrics on test
print(sklearn.metrics.classification_report(y_test,clf.predict(X_test)))

             precision    recall  f1-score   support

        0.0       0.56      0.04      0.07       801
        1.0       0.60      0.98      0.75      1199

avg / total       0.59      0.60      0.48      2000



## Random forest

Another possible classification routine we can use is a random forest (an ensemble of decision trees). This algorithm is very sensitive to overfitting a single sample, so I have carefully chosen a few non-default parameters:

In [41]:
#### Fit Random Forest
## Random Forests
from sklearn import ensemble
clf = ensemble.RandomForestClassifier(n_estimators=20, 
                                      max_depth=None,
                                      min_samples_split=200,
                                      random_state=0)
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

array([[ 800, 1634],
       [ 385, 3181]])

In [42]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,clf.predict(X_train))

0.66349999999999998

In [43]:
## Get kappa
sklearn.metrics.cohen_kappa_score(y_train,clf.predict(X_train))

0.24028341680557808

In [44]:
## get classification metrics
print(sklearn.metrics.classification_report(y_train,clf.predict(X_train)))

             precision    recall  f1-score   support

        0.0       0.68      0.33      0.44      2434
        1.0       0.66      0.89      0.76      3566

avg / total       0.67      0.66      0.63      6000



In [45]:
## get classification metrics on test
print(sklearn.metrics.classification_report(y_test,clf.predict(X_test)))

             precision    recall  f1-score   support

        0.0       0.55      0.27      0.36       801
        1.0       0.64      0.85      0.73      1199

avg / total       0.60      0.62      0.58      2000



## Score the new dataset (next year's data?)

Now that we have developed a model, we can use this model to score another dataset. In this case, we will import a similar dataset and generate predictions.

In [46]:
## Score the new dataset using this model
## Import Sales Data File
customer_sales_2 = pd.read_csv('data/simulated_sales_data/customer_sales2.csv')
## Change dtypes
customer_sales_2['customer_id'] = \
              customer_sales_2.customer_id.astype('object')
customer_sales_2['region'] = \
              customer_sales_2.region.astype('object')
customer_sales_2['current_customer'] = \
              customer_sales_2.current_customer.astype('category')
customer_sales_2['gender'] = \
              customer_sales_2.gender.astype('category')
customer_sales_2['activity'] = \
              pd.Categorical(customer_sales_2.activity,
                             ['Low','Med','High'],
                             ordered=True)
customer_sales_2['sale'] = \
              customer_sales_2.sale.astype('category')
Y,X = dmatrices(formula,
                customer_sales_2.dropna(),
                return_type="dataframe")

Now we can score the dataset using the model we just created:

In [47]:
customer_sales_2['Predicated_sale'] = \
                pd.Categorical.from_codes(clf.predict(X),
                                          categories=['No','Yes'])
pd.crosstab(customer_sales_2['Predicated_sale'],
            customer_sales_2['sale'])

sale              No   Yes
Predicated_sale           
No               216   190
Yes              591  1003